In [1]:
import cv2
import os
from matplotlib import pyplot as plt
from PIL import ImageFont, ImageDraw, Image, ImageColor
import numpy as np
import glob
import xmltodict
from tqdm import tqdm

In [2]:
chokepoint_root = "/home/oole/Data/Chokepoint"
def _load_roidb_chokepoint(subset):
    subset_path = os.path.join(chokepoint_root, "annotation", "G1", subset)
    paths = sorted(glob.glob(subset_path + "*/*/*/*"))
    vid_names_1 = [path for path in paths if "xml" not in path and "seq" in path]
    vid_names_1 = ["/".join(v.split("/")[-3:]) for v in vid_names_1]
    paths = sorted(glob.glob(subset_path + "*/*/*"))
    vid_names_2 = [path for path in paths if "xml" not in path and "seq" in path]
    vid_names_2 = ["/".join(v.split("/")[-2:]) for v in vid_names_2]
    vid_names = list(vid_names_2 + vid_names_1)
    return vid_names

def get_sequences(subset):
    subset_path = os.path.join(chokepoint_root, "annotation", "G1", subset)
    paths = sorted(glob.glob(subset_path + "*/*/*/*"))
    sequences_1 = [path for path in paths if "xml" not in path and "seq" in path]
    sequences_1 = ["/".join(v.split("/")[-3:]) for v in sequences_1]
    paths = sorted(glob.glob(subset_path + "*/*/*"))
    sequences_2 = [path for path in paths if "xml" not in path and "seq" in path]
    sequences_2 = ["/".join(v.split("/")[-2:]) for v in sequences_2]
    sequences = list(sequences_2 + sequences_1)
    return sequences

In [3]:
def get_sequence_frame_files(sequence, subset):
    sequence_folder = os.path.join(chokepoint_root, "annotation/G1/", subset, sequence)
    sequence_frame_files = sorted(glob.glob(sequence_folder + "/*.xml"))
    sequence_frame_numbers = [frame_file.split("/")[-1].split(".")[0] for frame_file in sequence_frame_files]
    first_frame_number = sequence_frame_numbers[0]
    return sequence_frame_files, first_frame_number

In [4]:
# Some helper functions to draw image with object boundary boxes
fontname = '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'
font = ImageFont.truetype(fontname, 40) if os.path.isfile(fontname) else ImageFont.load_default()

def bounding_box(img, xmin, ymin, xmax, ymax, width, score):
    draw = ImageDraw.Draw(img)
    xres, yres = img.size[0], img.size[1]
    box = [xmin, ymin, xmax, ymax]
    draw.rectangle(box, outline="red", width=width)

def plot_img(img, axes, xmin, ymin, xmax, ymax):
    for i in range(len(xmin)):
        bounding_box(img, xmin[i], ymin[i], xmax[i], ymax[i], 2, -1)
    plt.setp(axes, xticks=[], yticks=[])
    plt.imshow(img)


In [5]:
def obj_data_to_bbox(annotation):
    object_ann = annotation['object']
    bbox = object_ann['bndbox']
    x1 = int(bbox['xmin'])
    y1 = int(bbox['ymin'])
    x2 = int(bbox['xmax'])
    y2 = int(bbox['ymax'])
    box = [x1, y1, x2, y2]
    return box

def obj_file_path(annotation):
    sub_path = annotation['folder']
    fname = annotation['filename']
    full_path = os.path.join(chokepoint_root, sub_path, fname + ".jpg")
    return full_path

In [7]:
draw_gt_path = os.path.join(chokepoint_root, "sequences_drawn_gt")
dpi = 100
figsize = 800 / float(dpi), 600 / float(dpi)

sequences = get_sequences('train')
for sequence in tqdm(sequences):
    frame_files, first_frame_number = get_sequence_frame_files(sequence, 'train')
    for frame in frame_files:
        frame_dict =  xmltodict.parse(open(frame).read())['annotation']
        bbox = obj_data_to_bbox(frame_dict)
        bbox
        img_file_path = obj_file_path(frame_dict)
        drawn_gt_path = frame.replace("annotation", "sequences_drawn_gt").replace(".xml", ".jpg")
        
        if not os.path.isdir(os.path.dirname(drawn_gt_path)):
            os.makedirs(os.path.dirname(drawn_gt_path))
        
        read_image = Image.open(img_file_path)
        bounding_box(read_image, bbox[0], bbox[1], bbox[2], bbox[3], 5, -1)
        read_image.save(drawn_gt_path)
#         print(drawn_gt_path)


100%|██████████| 173/173 [00:50<00:00,  3.41it/s]
